In [1]:
from Utils.Solver import *
from Utils.Signals import *

In [ ]:
# Let's assume we are interested in the following stocks: AAPL, MSFT, TSLA, AMZN, GOOG
tickers = ['AAPL', 'MSFT', 'TSLA', 'AMZN', 'GOOG', 'IBM', 'NFLX', 'NVDA', 'AMD', 'INTC', 'TXN', 'QCOM', 'MU', 'ADI', 'HPQ', 'GLW', 'AVGO', 'ADP', 'INTU', 'PFE']

# Download historical stock data
data = yf.download(tickers, start='2000-01-01', end='2025-01-01')
# Initialize the portfolio solver with appropriate penalty and max weight threshold
portfolio_solver = Portfolio_Solver(0.8, max_weight_threshold=0.2 )

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  20 of 20 completed


In [3]:
start_date_signal = '2000-01-01'
end_date_signal = '2021-01-01'
date_range_signal = pd.date_range(start=start_date_signal, end=end_date_signal)

start_date_eval = '2019-01-01'
end_date_eval = '2020-01-01'
date_range_eval = pd.date_range(start=start_date_eval, end=end_date_eval)

In [4]:
# Initialize an empty list to store the rows for the first DataFrame
dataset_scores = []

# Step 1: Create the DataFrame with rsi_scores, macd_scores, and sma_scores
for date in date_range_signal:
    #print(f"processing {date}")

    # Step 1a: Calculate the RSI signal scores
    rsi_signal_scores = calculate_rsi_signal(data, tickers, date=date, period=14)
    rsi_scores = np.array([score[1] for score in rsi_signal_scores])

    # Step 1b: Calculate the MACD signal scores
    macd_signal_scores = calculate_macd_signal(data, tickers, date=date)
    macd_scores = np.array([score[1] for score in macd_signal_scores])

    # Step 1c: Calculate the SMA signal scores
    sma_signal_scores = calculate_sma_signal(data, tickers, date=date)
    sma_scores = np.array([score[1] for score in sma_signal_scores])

    rsi_avg = np.mean(rsi_scores)
    macd_avg = np.mean(macd_scores)
    sma_avg = np.mean(sma_scores)

    if np.any(np.isnan(rsi_scores)) or np.any(np.isnan(macd_scores)) or np.any(np.isnan(sma_scores)):
        #print(f"Skipping {date} due to NaN values in the signals.")
        continue  # Skip this date and move to the next one

    # Step 1d: Add the scores to the first dataset
    dataset_scores.append({
        'date': date,
        'rsi_scores': rsi_scores,
        'macd_scores': macd_scores,
        'sma_scores': sma_scores,
        'rsi_avg': rsi_avg,
        'macd_avg': macd_avg,
        'sma_avg': sma_avg
    })


# Convert the first dataset into a DataFrame
df_scores = pd.DataFrame(dataset_scores)
display(df_scores)

,date,rsi_scores,macd_scores,sma_scores,rsi_avg,macd_avg,sma_avg
0,2011-04-12,"[38.30985596937661, 45.976995148600494, 49.262...","[-0.041286021816696605, 0.07719779821613915, 0...","[1.2408266186714165, 0.3653725624084494, 0.015...",46.402125,0.003431,2.480006
1,2011-04-13,"[43.37429425160838, 45.7932061953191, 50.76935...","[-0.03208345698054789, 0.05630234146250271, 2....","[1.22343857049942, 0.3209991168975854, 0.01655...",48.389809,-0.005745,2.453246
2,2011-04-14,"[39.868432186283734, 41.99392283155413, 51.922...","[-0.031152578633931824, 0.03080863210926836, -...","[1.204104969501497, 0.2738248157501211, 0.0177...",48.587490,-0.008806,2.426521
3,2011-04-15,"[35.71245263444316, 41.119200007163236, 54.335...","[-0.03778464009489921, 0.011577156210363121, -...","[1.182603130340576, 0.23108903884887866, 0.019...",48.959229,-0.008776,2.398987
4,2011-04-18,"[41.52281544579322, 36.38519414873376, 50.8965...","[-0.03076641075297963, -0.014564581044156213, ...","[1.161009483337402, 0.18357344627380456, 0.019...",46.174009,-0.018784,2.364460
...,...,...,...,...,...,...,...
2443,2020-12-24,"[67.920937118102, 61.41404377157558, 62.871929...","[0.6376303571044013, 0.8196496588581172, -1.10...","[20.276957168579102, 16.26274291992189, 64.366...",56.104713,-0.217209,17.039354
2444,2020-12-28,"[73.64743031225844, 64.3647220972155, 63.14165...","[0.87741210428235, 0.8857953923217612, -1.1634...","[20.262904510498046, 16.051133270263676, 64.87...",57.368075,-0.205486,17.001680
2445,2020-12-29,"[68.56489172441627, 62.478945630153646, 63.483...","[0.8423498299198462, 0.8160749698159582, -1.22...","[20.211753349304203, 15.715765304565451, 65.42...",56.378226,-0.194751,16.928631
2446,2020-12-30,"[65.48952010409074, 56.99503411626423, 67.5459...","[0.6767051937654904, 0.5607928933321322, -0.71...","[20.21596124649048, 15.50270141601564, 66.1669...",56.857889,-0.193543,16.915374


In [5]:
start_date_train = '2018-01-01'
end_date_train = '2019-01-01'
date_range_train = pd.date_range(start=start_date_train, end=end_date_train)

start_date_eval = '2018-01-01'
end_date_eval = '2019-01-01'
date_range_eval = pd.date_range(start=start_date_eval, end=end_date_eval)

# Filter the dataframe within the date range
df_train = df_scores[(df_scores['date'] >= start_date_train) & (df_scores['date'] <= end_date_train)]
display(df_train)
df_eval = df_scores[(df_scores['date'] >= start_date_eval) & (df_scores['date'] <= end_date_eval)]
display(df_eval)

,date,rsi_scores,macd_scores,sma_scores,rsi_avg,macd_avg,sma_avg
1692,2018-01-02,"[52.03591507999248, 59.83768857477732, 48.9947...","[-0.11910990770038687, -0.0818355949544598, -0...","[3.489654769897463, 9.188492984771727, -0.8016...",57.742113,0.033368,5.465428
1693,2018-01-03,"[51.95032807281302, 62.07820508779738, 46.7644...","[-0.09802692156766507, -0.05686884634409706, -...","[3.5293139076232904, 9.231407546997076, -0.857...",61.618370,0.112901,5.488661
1694,2018-01-04,"[54.11768205067581, 65.96358293926005, 44.9957...","[-0.06944995190564734, -0.0006591077725789374,...","[3.570286369323725, 9.281218872070312, -0.9083...",63.603041,0.184443,5.517775
1695,2018-01-05,"[59.01943931811351, 70.5770397545288, 46.61619...","[-0.01997081745081536, 0.09107108660953012, -0...","[3.615682468414306, 9.349188251495363, -0.9565...",65.278649,0.250290,5.549686
1696,2018-01-08,"[56.861145908548075, 70.93065789419548, 59.587...","[0.0007720159179399055, 0.1416295707563553, 0....","[3.6614380645751936, 9.42191036224365, -0.9697...",66.155930,0.291734,5.593833
...,...,...,...,...,...,...,...
1938,2018-12-24,"[22.983628148431805, 29.47614862834661, 31.334...","[-0.27370507608551664, -1.2587347144455032, -0...","[-0.38992567062378214, 3.6595485305786184, 1.2...",30.627680,-0.698724,-3.311387
1939,2018-12-26,"[36.492562846045836, 43.112559059436315, 45.44...","[-0.14586524619071817, -1.0181984872209404, -0...","[-0.6690888404846191, 3.4754394531249915, 1.31...",42.217574,-0.527613,-3.486970
1940,2018-12-27,"[35.82502728845707, 44.232293165358904, 42.397...","[-0.05313712519457159, -0.7668437667595509, -0...","[-0.9313852310180621, 3.3260189437866217, 1.39...",43.736315,-0.338804,-3.638528
1941,2018-12-28,"[35.924029099343514, 43.06904043290133, 48.954...","[0.029587227237728886, -0.604779790741828, -0....","[-1.2179713249206543, 3.099160690307613, 1.471...",44.431080,-0.186764,-3.835807


,date,rsi_scores,macd_scores,sma_scores,rsi_avg,macd_avg,sma_avg
1692,2018-01-02,"[52.03591507999248, 59.83768857477732, 48.9947...","[-0.11910990770038687, -0.0818355949544598, -0...","[3.489654769897463, 9.188492984771727, -0.8016...",57.742113,0.033368,5.465428
1693,2018-01-03,"[51.95032807281302, 62.07820508779738, 46.7644...","[-0.09802692156766507, -0.05686884634409706, -...","[3.5293139076232904, 9.231407546997076, -0.857...",61.618370,0.112901,5.488661
1694,2018-01-04,"[54.11768205067581, 65.96358293926005, 44.9957...","[-0.06944995190564734, -0.0006591077725789374,...","[3.570286369323725, 9.281218872070312, -0.9083...",63.603041,0.184443,5.517775
1695,2018-01-05,"[59.01943931811351, 70.5770397545288, 46.61619...","[-0.01997081745081536, 0.09107108660953012, -0...","[3.615682468414306, 9.349188251495363, -0.9565...",65.278649,0.250290,5.549686
1696,2018-01-08,"[56.861145908548075, 70.93065789419548, 59.587...","[0.0007720159179399055, 0.1416295707563553, 0....","[3.6614380645751936, 9.42191036224365, -0.9697...",66.155930,0.291734,5.593833
...,...,...,...,...,...,...,...
1938,2018-12-24,"[22.983628148431805, 29.47614862834661, 31.334...","[-0.27370507608551664, -1.2587347144455032, -0...","[-0.38992567062378214, 3.6595485305786184, 1.2...",30.627680,-0.698724,-3.311387
1939,2018-12-26,"[36.492562846045836, 43.112559059436315, 45.44...","[-0.14586524619071817, -1.0181984872209404, -0...","[-0.6690888404846191, 3.4754394531249915, 1.31...",42.217574,-0.527613,-3.486970
1940,2018-12-27,"[35.82502728845707, 44.232293165358904, 42.397...","[-0.05313712519457159, -0.7668437667595509, -0...","[-0.9313852310180621, 3.3260189437866217, 1.39...",43.736315,-0.338804,-3.638528
1941,2018-12-28,"[35.924029099343514, 43.06904043290133, 48.954...","[0.029587227237728886, -0.604779790741828, -0....","[-1.2179713249206543, 3.099160690307613, 1.471...",44.431080,-0.186764,-3.835807


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

# Convert your training features to torch tensor
X_train = torch.tensor(df_train[['rsi_avg', 'macd_avg', 'sma_avg']].values, dtype=torch.float32)
signal_data = df_train[['rsi_scores', 'macd_scores', 'sma_scores']].values  # shape: (N, 3), each cell is a vector

# Initialize weights
signal_weights = torch.nn.Parameter(torch.tensor([1/3, 1/3, 1/3], dtype=torch.float32), requires_grad=True)

# Optimizer
optimizer = optim.Adam([signal_weights], lr=0.01)

# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    total_return = 0.0
    batch_returns = []

    for idx in range(len(df_train)):
        rsi_scores = df_train.iloc[idx]['rsi_scores']
        macd_scores = df_train.iloc[idx]['macd_scores']
        sma_scores = df_train.iloc[idx]['sma_scores']
        
        # Combine signals using current weights (detached to use numpy for the solver)
        combined_scores = combine_signals2(signal_weights.detach().numpy(), [rsi_scores, macd_scores, sma_scores])
        
        # Get price data at current date
        date = df_train.iloc[idx]['date']
        window_start = date - pd.Timedelta(days=30)
        price_data = data.loc[window_start:date]

        # Skip if data is too short or empty
        if price_data.empty or len(price_data) < 2:
            continue

        
        # Solve portfolio
        weights = portfolio_solver.SolveSignalPortfolioMVO(tickers, price_data, combined_scores)

        # Compute return using weights
        _, ret, _ = portfolio_solver.CalculatePortfolioReturns(tickers, price_data, weights, start_date=date.strftime('%Y-%m-%d'), time_period=1)
        
        batch_returns.append(ret)

    # Compute mean return as reward
    mean_return = np.mean(batch_returns)

    # Manual gradient step: use return as loss (maximize)
    optimizer.zero_grad()
    loss = -torch.tensor(mean_return, dtype=torch.float32)
    loss.backward()  # This causes error — so instead we skip .backward()

    # Manual gradient-free step (REINFORCE-like)
    with torch.no_grad():
        noise = torch.randn_like(signal_weights)
        lr = 0.05
        signal_weights += lr * mean_return * noise
        signal_weights[:] = torch.clamp(signal_weights, 0, 1)
        signal_weights[:] = signal_weights / torch.sum(signal_weights)  # Normalize to sum to 1

    print(f"[Epoch {epoch+1}] Mean Return: {mean_return:.5f}, Signal Weights: {signal_weights.detach().numpy()}")


     pcost       dcost       gap    pres   dres
 0: -6.4352e-02 -4.2443e+00  4e+00  4e-16  5e-16
 1: -6.6118e-02 -2.7945e-01  2e-01  1e-16  8e-16
 2: -8.6205e-02 -1.2212e-01  4e-02  2e-16  9e-17
 3: -9.7679e-02 -1.0185e-01  4e-03  1e-16  9e-17
 4: -9.9274e-02 -9.9927e-02  7e-04  2e-16  1e-16
 5: -9.9584e-02 -9.9593e-02  9e-06  2e-16  6e-17
 6: -9.9589e-02 -9.9589e-02  9e-08  2e-16  8e-17
Optimal solution found.


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Convert your score dataframe to list of daily signals
signal_data = df_train[['rsi_scores', 'macd_scores', 'sma_scores']].to_dict('records')

# Convert to torch tensors
signal_weights = torch.tensor([1/3, 1/3, 1/3], requires_grad=True, dtype=torch.float32)
optimizer = optim.Adam([signal_weights], lr=0.01)

# Define the loss function as the negative return (we want to maximize return)
def compute_loss(total_return):
    return -total_return  # Maximize return -> minimize negative return

# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    optimizer.zero_grad()

    total_returns = []

    for index, row in df_train.iterrows():
        rsi = np.array(row['rsi_scores'])
        macd = np.array(row['macd_scores'])
        sma = np.array(row['sma_scores'])
        
        # Normalize signals (within each day)
        def normalize(v):
            v = np.array(v)
            return (v - v.min()) / (v.max() - v.min() + 1e-8)
        
        rsi_norm = normalize(rsi)
        macd_norm = normalize(macd)
        sma_norm = normalize(sma)

        # Convert signal tensors
        rsi_t = torch.tensor(rsi_norm, dtype=torch.float32)
        macd_t = torch.tensor(macd_norm, dtype=torch.float32)
        sma_t = torch.tensor(sma_norm, dtype=torch.float32)

        # Combine signals with trainable weights
        combined_signal = signal_weights[0] * rsi_t + signal_weights[1] * macd_t + signal_weights[2] * sma_t
        combined_signal = combined_signal.detach().numpy()  # Detach for CVXOPT

        # Use the optimizer with combined signal
        date = row['date']
        window_data = data.loc[:str(date)].iloc[-252:]  # trailing year of data
        try:
            weights = portfolio_solver.SolveSignalPortfolioMVO(tickers, window_data, combined_signal)
            _, total_return, _ = portfolio_solver.CalculatePortfolioReturns(tickers, window_data, weights)
            total_returns.append(total_return)
            print("total RETURNS : ", total_returns)
        except:
            continue  # skip if solver fails

    if total_returns:
        mean_return = np.mean(total_returns)
        loss = compute_loss(torch.tensor(mean_return, dtype=torch.float32))
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            signal_weights.clamp_(0, 1)  # optional: keep weights positive
            signal_weights /= signal_weights.sum()  # normalize to sum to 1

        print(f"Epoch {epoch+1}/{num_epochs}, Mean Return: {mean_return:.5f}, Loss: {loss.item():.5f}")
        print(f"Current Weights: {signal_weights.detach().numpy()}")


     pcost       dcost       gap    pres   dres
 0: -6.3340e-02 -4.2417e+00  4e+00  3e-17  4e-16
 1: -6.5117e-02 -2.7957e-01  2e-01  1e-16  7e-16
 2: -8.5020e-02 -1.2091e-01  4e-02  2e-16  1e-16
 3: -9.6850e-02 -1.0125e-01  4e-03  2e-16  8e-17
 4: -9.8739e-02 -9.9012e-02  3e-04  2e-16  8e-17
 5: -9.8860e-02 -9.8862e-02  3e-06  2e-16  8e-17
 6: -9.8861e-02 -9.8861e-02  3e-08  2e-16  9e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -6.0223e-02 -4.2452e+00  4e+00  2e-16  5e-16
 1: -6.1519e-02 -2.6868e-01  2e-01  2e-16  6e-16
 2: -7.8742e-02 -1.0444e-01  3e-02  1e-16  1e-16
 3: -8.6264e-02 -9.5139e-02  9e-03  2e-16  5e-17
 4: -8.7680e-02 -9.0976e-02  3e-03  2e-16  5e-17
 5: -8.8805e-02 -8.9434e-02  6e-04  2e-16  1e-16
 6: -8.9056e-02 -8.9064e-02  9e-06  2e-16  6e-17
 7: -8.9060e-02 -8.9060e-02  9e-08  1e-16  7e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -5.9155e-02 -4.2483e+00  4e+00  2e-16  5e-16
 1: -6.0229e-02 -2.5781e

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
class SignalWeightModel(nn.Module):
    def __init__(self):
        super(SignalWeightModel, self).__init__()
        # Raw weights — no constraints yet
        self.raw_weights = nn.Parameter(torch.tensor([1.0, 1.0, 1.0], dtype=torch.float32))

    def forward(self):
        # Normalize weights so they sum to 1
        normalized_weights = torch.softmax(self.raw_weights, dim=0)
        return normalized_weights

In [ ]:
def combine_signals_torch(signal_weights, signal_scores):
    rsi, macd, sma = signal_scores

    def normalize_to_01_torch(values):
        min_val = torch.min(values)
        max_val = torch.max(values)
        if (max_val - min_val).item() == 0:
            return torch.zeros_like(values)
        return (values - min_val) / (max_val - min_val)

    rsi = torch.tensor(rsi, dtype=torch.float32)
    macd = torch.tensor(macd, dtype=torch.float32)
    sma = torch.tensor(sma, dtype=torch.float32)

    rsi_norm = normalize_to_01_torch(rsi)
    macd_norm = normalize_to_01_torch(macd)
    sma_norm = normalize_to_01_torch(sma)

    combined = rsi_norm * signal_weights[0] + macd_norm * signal_weights[1] + sma_norm * signal_weights[2]
    combined = torch.clamp(combined, -5, 5)

    return combined.detach().numpy()  # Convert back to NumPy for optimizer

In [ ]:
# Setup
model = SignalWeightModel()
optimizer = optim.Adam(model.parameters(), lr=0.1)
num_epochs = 50

for epoch in range(num_epochs):
    total_return_all_days = []

    for _, row in df_train.iterrows():
        rsi = row['rsi_scores']
        macd = row['macd_scores']
        sma = row['sma_scores']

        signal_weights = model()

        # Combine signals using current weights
        combined_scores = combine_signals_torch(signal_weights, (rsi, macd, sma))

        # Portfolio optimization (no gradients inside)
        weights = portfolio_solver.SolveSignalPortfolioMVO(tickers, data, combined_scores)

        # Portfolio return (no gradients inside)
        _, total_return, _ = portfolio_solver.CalculatePortfolioReturns(
            tickers=tickers,
            data=data,
            weights=weights,
            start_date=str(row['date'].date()),
            time_period=225
        )

        total_return_all_days.append(total_return)

    # Compute loss: negative mean return (we want to maximize return)
    mean_return = np.mean(total_return_all_days)
    loss = -torch.tensor(mean_return, requires_grad=True)

    # Backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Mean Return: {mean_return:.5f}, Loss: {loss.item():.5f}")

# Final optimized weights
final_weights = model().detach().numpy()
print("Optimized Signal Weights:", final_weights)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

class SignalWeightModel(nn.Module):
    def __init__(self):
        super(SignalWeightModel, self).__init__()
        self.raw_weights = nn.Parameter(torch.tensor([1.0, 1.0, 1.0], dtype=torch.float32))

    def forward(self):
        return torch.softmax(self.raw_weights, dim=0)

def normalize_to_01_torch(values):
    min_val = torch.min(values)
    max_val = torch.max(values)
    if (max_val - min_val).item() == 0:
        return torch.zeros_like(values)
    return (values - min_val) / (max_val - min_val)

def combine_signals_torch(signal_weights, rsi, macd, sma):
    rsi_norm = normalize_to_01_torch(rsi)
    macd_norm = normalize_to_01_torch(macd)
    sma_norm = normalize_to_01_torch(sma)
    combined = signal_weights[0]*rsi_norm + signal_weights[1]*macd_norm + signal_weights[2]*sma_norm
    return combined

model = SignalWeightModel()
optimizer = optim.Adam(model.parameters(), lr=0.1)
num_epochs = 50

for epoch in range(num_epochs):
    returns = []

    for _, row in df_train.iterrows():
        rsi = torch.tensor(row['rsi_scores'], dtype=torch.float32)
        macd = torch.tensor(row['macd_scores'], dtype=torch.float32)
        sma = torch.tensor(row['sma_scores'], dtype=torch.float32)

        signal_weights = model()
        combined_signal = combine_signals_torch(signal_weights, rsi, macd, sma)
        portfolio_weights = combined_signal / combined_signal.sum()  # Simulated weights

        # Get 1-day return from next day close
        date = str(row['date'].date())
        data_returns = data['Close'].pct_change().dropna()
        try:
            day_index = data_returns.index.get_loc(pd.to_datetime(date))
            next_day = data_returns.iloc[day_index + 1][tickers].values
        except:
            continue

        daily_return = torch.tensor(next_day, dtype=torch.float32) @ portfolio_weights
        returns.append(daily_return)

    if not returns:
        continue

    returns = torch.stack(returns)
    mean_return = returns.mean()
    loss = -mean_return  # Maximize mean return

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Mean Return: {mean_return.item():.5f}, Loss: {loss.item():.5f}")

final_weights = model().detach().numpy()
print("✅ Optimized Signal Weights:", final_weights)


In [ ]:
average_annualized_return1, total_return_sum1 = portfolio_solver.CalculateEvalReturns(tickers, data, df_eval, final_weights)

average_annualized_return2, total_return_sum2 = portfolio_solver.CalculateEvalReturns(tickers, data, df_eval, [1,1,1])


In [ ]:
print(f"average ann ret : {average_annualized_return1}, total ret sum : {total_return_sum1}")
print(f"1/n average ann ret : {average_annualized_return2}, total ret sum : {total_return_sum2}")

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from cvxpylayers.torch import CvxpyLayer
import cvxpy as cp
import numpy as np

class DifferentiableMVO(nn.Module):
    def __init__(self, n_assets, max_weight=0.2, risk_aversion=0.5):
        super().__init__()
        self.n_assets = n_assets
        self.max_weight = max_weight
        self.risk_aversion = risk_aversion

        # Define the optimization problem
        w = cp.Variable(n_assets)
        mu = cp.Parameter(n_assets)  # expected returns
        Sigma = cp.Parameter((n_assets, n_assets))  # covariance
        scores = cp.Parameter(n_assets)

        objective = cp.Maximize(mu @ w - risk_aversion * cp.quad_form(w, Sigma) + scores @ w)
        constraints = [
            cp.sum(w) == 1,
            w >= 0,
            w <= max_weight
        ]

        problem = cp.Problem(objective, constraints)
        self.layer = CvxpyLayer(problem, parameters=[mu, Sigma, scores], variables=[w])

    def forward(self, mu, Sigma, scores):
        weights, = self.layer(mu, Sigma, scores)
        return weights


# Learnable signal weights
signal_weights = nn.Parameter(torch.tensor([1/3, 1/3, 1/3], dtype=torch.float32), requires_grad=True)

# Training setup
optimizer = Adam([signal_weights], lr=0.01)
loss_fn = lambda returns: -returns.mean()  # Maximize average return

# Mock your data (replace with your real data loop)
for epoch in range(50):
    total_returns = []
    for i in range(len(df_train)):
        rsi = torch.tensor(df_train.iloc[i]['rsi_scores'], dtype=torch.float32)
        macd = torch.tensor(df_train.iloc[i]['macd_scores'], dtype=torch.float32)
        sma = torch.tensor(df_train.iloc[i]['sma_scores'], dtype=torch.float32)

        # Normalize signals
        def normalize(x): return (x - x.min()) / (x.max() - x.min() + 1e-8)
        rsi, macd, sma = normalize(rsi), normalize(macd), normalize(sma)

        combined_scores = rsi * signal_weights[0] + macd * signal_weights[1] + sma * signal_weights[2]

        # Get return stats (replace with your actual historical mu and Sigma for this date)
        mu_np = var_data['Close'].pct_change().dropna().mean().values.astype(np.float32)
        Sigma_np = var_data['Close'].pct_change().dropna().cov().values.astype(np.float32)

        mu = torch.tensor(mu_np, dtype=torch.float32)
        Sigma = torch.tensor(Sigma_np, dtype=torch.float32)

        mvo = DifferentiableMVO(len(tickers))
        weights = mvo(mu, Sigma, combined_scores)

        returns = (mu @ weights).sum()  # dot product as expected portfolio return
        total_returns.append(returns)

    total_returns = torch.stack(total_returns)
    loss = loss_fn(total_returns)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        normalized_weights = signal_weights / signal_weights.sum()

    print(f"Epoch {epoch+1}/50, Avg Return: {total_returns.mean().item():.5f}, Signal Weights: {normalized_weights.detach().numpy()}")
